In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [29]:
class KNNRegression:
    def __init__ (self, X, Y, k):
        self.k = k
        self.X = X
        self.Y = Y
    
    def EuclidieanDistance (self, x1):
        if self.train == True:
            self.x2 = self.X[self.idx]
        else: 
            self.x2 = self.x_test
        distance = np.sqrt(np.sum(np.square(x1-self.x2)))
        return distance
    
    def calc_neighbors(self):
        costs = np.apply_along_axis(self.EuclidieanDistance, 1, self.X)
        sorted_costs = np.sort(costs)
        indices = np.where(costs <= sorted_costs[self.k-1])[0]
        return indices
    
    def predictions(self, indices):
        prediction = float(0)
        for i in indices:
            prediction += self.Y[i]
        prediction = prediction / indices.shape
        return float(prediction)
    
    def predict(self, x):
        self.train = False
        self.x_test = x
        indices =  self.calc_neighbors()
        print(indices)
        return self.predictions(indices)
    
    def accuracy(self, x_test_set, y_test_set):
        y_pred = []
        for i in  range(x_test_set.shape[0]):
            self.train = False
            self.x_test = x_test_set[i]
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        ssr = np.sum((y_pred - y_test_set)**2)
        sst = np.sum((y_test_set - y_test_set.mean())**2)
        self.r2_score = 1 - (ssr/sst)
        return self.r2_score, y_pred
        
    def propogate(self):
        y_pred = []
        self.train = True
        for i in range(self.X.shape[0]):
            self.idx = i
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        print("Model Trained")

In [30]:
a = np.array([[1, 1, 1], [3, 3, 3], [2, 2, 2]])

b = np.array([1, 3, 2])
print(a.shape, b.shape)
a = KNNRegression(a, b, 2)
a.propogate()
temp_1 = np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]])
temp_2 = np.array([[3.1], [2.1]])
print(temp_1.shape, temp_2.shape)
a.accuracy(np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]]), np.array([[3.1], [2.1]]))

(3, 3) (3,)
Model Trained
(2, 3) (2, 1)


(-1.08, [2.5, 2.5])

In [31]:
a.predict(np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]]))

[1 2]


2.5

In [32]:
# Data Preprocessing

# Changing all yes/no data to 0/1, normalization of price and area, and outlier removal 
df = pd.read_csv('Housing.csv')
for column in df:  
    df[column].replace(('yes','no'), (0,1), inplace = True)
    if(column == 'area'):
        df[column] = (df[column] - df[column].mean()) / df[column].std()
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df.drop(df[(df[column] < Q1-1.5*IQR) | (df[column] > Q3+1.5*IQR)].index, inplace = True) 
        
# Adding One Hot Encoding to furnishingstatus 

df = pd.get_dummies(df, columns = ['furnishingstatus']) 

df.head()
df.shape

(530, 15)

In [33]:
Y = np.array(df.iloc[:, 0:1])
print(Y.shape)
X = np.array(df.iloc[:, 1:])
print(X.shape)

(530, 1)
(530, 14)


In [34]:
def split_data(X, Y, test_size=0.1, random_state=42):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = 42)
    x_train = np.array(x_train.T)
    x_test = np.array(x_test.T)
    y_train = np.array(y_train.T)
    y_test = np.array(y_test.T)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test  = split_data(X, Y)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(14, 477) (14, 53) (1, 477) (1, 53)


In [37]:
model = KNNRegression(x_train, y_train.T, 5)
model.propogate()
r2_score, predictions = model.accuracy(x_train, y_train.T)
print(r2_score)S

Model Trained
-17.004419699597342
